In [880]:
#author - Ashish Kumar Jayant

In [1147]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import math
import itertools

In [1148]:
file = pd.read_excel("file.xlsx",sheetname='sheet1')

In [1149]:
def make_cat(file):
    file['y1_category'] = file['Y1'].map(lambda x: 0 if (x>6.0) & (x<12) else (1 if((x>12) & (x<28))  else 2))
    file['y1_good'] = file['Y1'].map(lambda x: 1 if (x>6) & (x<14.5) else 0)
    file['y1_average'] = file['Y1'].map(lambda x: 1 if (x>14.5) & (x<28) else 0)
    file['y1_bad'] = file['Y1'].map(lambda x: 1 if (x>28) & (x<44) else 0)
    return file

In [1150]:
def make_x_y(file):
    x_train_full = file.loc[:,'X1': 'X8']/file.loc[:,'X1':'X8'].max()
    x_train = np.array(x_train_full[:int(0.7*len(x_train_full))])
    y1_train_full = np.array(file.loc[:,'y1_good':'y1_bad'])
    y1_train = np.array(y1_train_full[:int(0.7*len(y1_train_full))])
    return x_train,y1_train

In [1151]:
file = make_cat(file)

#### Features (normalized)

In [1152]:
pd.DataFrame(x_train).head()

,0,1,2,3,4,5,6,7
0,0.775510,0.818182,1.000000,0.555556,1.0,1.0,1.000,1.0
1,0.632653,1.000000,0.882353,1.000000,0.5,0.4,1.000,0.8
2,0.673469,0.939394,0.764706,1.000000,0.5,0.6,0.625,0.8
3,0.918367,0.696970,0.764706,0.555556,1.0,0.6,0.250,0.6
4,0.836735,0.757576,0.764706,0.666667,1.0,0.4,0.250,0.6


#### Targets

In [1153]:
pd.DataFrame(y1_train).head()

,0,1,2
0,0,0,1
1,0,1,0
2,1,0,0
3,0,0,1
4,0,1,0


In [1168]:
def network_structure(structure):
    layers = len(structure)
    biases = [np.random.rand(y,1) for y in structure[1:]]
    weights = [np.random.rand(x,y) for y,x in zip(structure[:-1],structure[1:]) ]
    return layers,weights,biases         

In [1155]:
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

In [1156]:
def relu(x):
    return np.log(1+np.exp(-x))

In [1157]:
def relu_diff(x):
    return 1/(1+np.exp(-x))

In [1158]:
def sigmoid_diff(x):
    return sigmoid(x)*(1-sigmoid(x))

In [1238]:
def feedforward(x_input,weights,biases):
    z = [0]*layers
    z[0] = x_input
    for i in range(1,layers):
        z[i] = (sigmoid(np.matmul(weights[i-1],z[i-1])))#+ biases[i-1]))
    z = [arr.reshape(arr.shape[0],1) for arr in z]
    for i in range(1,layers):
        z[i]+=biases[i-1]
    return z        

In [1239]:
def accuracy(y1_train,pred):
    count = 0
    c = [i for i in y1_train - pred]
    for j in c:
        if (np.count_nonzero(j) == 0):
            count+=1
    return(count/len(x_train))

#### cost function - cross entropy

In [1240]:
def cost_derivative_ce(output_activations,y,m):
    return np.sum(((y/output_activations - (1-y)/(1-output_activations))))/m           

#### cost function -  mean squared error

In [1241]:
def cost_derivative_mse(output_activations,y,m):
    return np.sum((np.abs(y-output_activations)))/m           

In [1242]:
def backprop(y,m,z,weights,lr):
    grad_cw = [0]*(layers-1)
    grad_cb = [0]*(layers-1)
    prod_1 = 1
    for i in range(layers-1,0,-1):
        prod = 1
        if(prod_1==1):
            prod *= cost_derivative_ce(z[layers-1],y,m)
        prod = prod*prod_1*sigmoid_diff(z[i])
        prod *= sigmoid_diff(np.dot(weights[i-1],z[i-1]))
        prod1 = prod
        #print("---prod.shape ",prod,"-----z.shape",z)
        try:
            prod2 = prod*weights[i-1]*z[i-1]
        except:
            prod2 = prod.T*z[i-1]
        grad_cw[i-1] = lr*prod2
        grad_cb[i-1] = lr*prod
    return grad_cw,grad_cb

In [1243]:
def run_this(epochs,file,weights,biases,z):
    for j in range(epochs):
        #file = shuffle(file)
        #x_train,y1_train = make_x_y(file)
        for i in range(len(x_train)):
            x_input=x_train[i]
            y1 = y1_train[i]
            z = feedforward(x_input,weights,biases)
            #z = [arr.reshape(arr.shape[0],1) for arr in z]
            delta_cw,delta_cb= backprop(y1,537,z,weights,learning_rates)       
            weights = np.add(weights,[arr.T for arr in delta_cw])
            biases = np.add(biases,delta_cb)

### ------------------------------------------------------------------------------------------------------------------

In [1244]:
file = shuffle(file)
x_train,y1_train = make_x_y(file)

In [1245]:
layers,weights,biases = network_structure([8,5,5,3]) 

In [1246]:
learning_rates = 0.9 
x_input = x_train[0]
z = feedforward(x_input,weights,biases)


In [1247]:
run_this(1000,file,weights,biases,z)

#### Making predictions

In [1248]:
pred = []
for i in x_train:
    t = feedforward(i,weights,biases)
    pred.append(t[-1])

In [1249]:
##pred_scores
pred[:5]

[array([[ 1.46486477],
        [ 1.81013306],
        [ 1.13606875]]), array([[ 1.46488086],
        [ 1.81014668],
        [ 1.13608151]]), array([[ 1.46468896],
        [ 1.80997285],
        [ 1.13592205]]), array([[ 1.4645344 ],
        [ 1.80977379],
        [ 1.13572738]]), array([[ 1.46478288],
        [ 1.81000539],
        [ 1.13594114]])]

In [1250]:
# making max prediction score 1 and rest 0 for classificaton
for i in range(len(pred)):
    x = np.argmax(pred[i])
    pred[i] = np.zeros(3)
    pred[i][x] = 1
    

In [1251]:
pred[:5]

[array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.])]

In [1252]:
y1_train[:5]

array([[1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [1253]:
accuracy(y1_train,pred)

0.3016759776536313